<img src="../.images/logosnb.png" alt="Banner" style="width: 800px;"/>

<div style='color: #690027;' markdown="1">
    <h1>DETECTIE VAN HUIDMONDJES</h1> 
</div>

<div class="alert alert-box alert-success">
In deze notebook wordt een diep neuraal netwerk gebruikt om stomata op een grote afbeelding te herkennen.
</div>

<div class="alert alert-block alert-warning"> 
Hoe een diep neuraal netwerk werkt en wat de onderdelen zijn kan je terugvinden in de notebook ['Fundamenten van een diep neuraal netwerk voor beeldherkenning'](../IntroductieDeepLearning/BasisDiepNeuraalNetwerk.ipynb). 
</div>

Voer onderstaande code-cel uit om van de functies in deze notebook gebruik te kunnen maken.

In [ ]:
import imp
with open('../.scripts/diep_neuraal_netwerk.py', 'rb') as fp:
    diep_neuraal_netwerk = imp.load_module('.scripts', fp, '../.scripts/diep_neuraal_netwerk.py', ('.py', 'rb', imp.PY_SOURCE))

<div style='color: #690027;' markdown="1">
    <h2>1. Het referentienetwerk</h2> 
</div>

Het netwerk waarvoor we hebben gekozen bestaat uit 3 convolutionele lagen met respectievelijk 32, 64 en 128 filters en heeft 1 feedforward laag met 64 neuronen. Het gebruikt stochastic gradient descent als optimizer met een learning rate van 0,01. Het netwerk gebruikt ook dezelfde regularisatie technieken die beschreven zijn in ['Overfitting'](../IntroductieDeepLearning/Overfitting.ipynb).

De volgende afbeelding geeft dit netwerk weer zoals de netwerken die we gezien hebben in ['Fundamenten van een diep neuraal netwerk voor beeldherkenning'](../IntroductieDeepLearning/BasisDiepNeuraalNetwerk.ipynb).

<img src="../.images/IntroductieDeepLearning/referentie_netwerk.png"/>

Om dit netwerk in te laden voer je de volgende code-cel uit:

In [ ]:
diep_neuraal_netwerk.laad_referentie_model()

<div style='color: #690027;' markdown="1">
    <h2>2. Stomatadetectie op een microfoto van een blad</h2> 
</div>

Momenteel kan het getrainde netwerk een classificatie maken tussen 2 klassen ("Stoma" of "Geen stoma") wanneer het een afbeelding van 120 pixels op 120 pixels krijgt als invoer. We willen echter dat alle stomata in een grote afbeelding (1600 px op 1200 px) gevonden kunnen worden met ons netwerk. Hierbij te verstaan dat de stomata ongeveer 120 op 120 pixels groot zijn op de foto. Om dit te verwezenlijken maken we gebruik van een <b>sliding window</b>. Dit is een vierkantje van 120 px op 120 px dat uit de grote afbeelding wordt geknipt en als invoer van het getrainde netwerk wordt gebruikt. Voor dit vierkantje zal het netwerk dan een voorspelling geven waarna het vierkantje naar rechts opschuift met 10 pixels. Wanneer het vierkantje de volledige breedte van de afbeelding heeft doorlopen zal het opnieuw naar de linkerkant verspringen maar dan 10 pixels lager dan de vorige keer. Dit proces herhaalt zich tot de volledige afbeelding verwerkt werd met telkens sprongen van 10 pixels. Volgende afbeelding geeft een illustratie, rechts van de afbeelding zie je de voorspelling van het netwerk. Een voorspelling van '1' wil zeggen dat het netwerk 100% zeker is dat er een stoma in het vierkantje zit, een voorspelling van '0' wil zeggen dat het netwerk oordeelt dat er zeker geen stoma in het vierkantje zit. Zodra de waarde groter wordt dan 0.5 (de drempelwaarde: zie later) zal het vierkantje geel worden.

<img src="../.images/IntroductieDeepLearning/sliding_window.gif"/>

Voor elke vierkantje uit de grote afbeelding zal er op deze manier een voorspelling gemaakt worden. Vaak zullen er meerdere vierkantjes rond de effectieve stoma een positieve uitvoer geven ("Stoma") omdat er slechts met 10 pixels wordt opgeschoven en dus niet veel verschil in de vierkantjes zit. Om deze vele positieve voorspellingen samen te voegen, wordt er gebruik gemaakt van <b>clustering</b>. Clustering zal punten die dicht bij elkaar liggen als één cluster beschouwen en het middelpunt van deze clusters is dan het gevonden stoma. Er bestaan verschillende methoden om clustering uit te voeren, wij maken gebruik van <b>mean shift clustering</b>. Bij mean shift clustering zal elk punt iteratief worden verplaatst in de richting waar de punten het dichtst bij elkaar liggen binnen een bepaalde straal (deze straal wordt ook wel de bandwidth genoemd). Uiteindelijk zullen de punten niet (of nauwelijks) meer verplaatsen, de punten die dan op dezelfde plek zijn terechtgekomen, behoren tot dezelfde cluster. Volgende afbeelding geeft een voorbeeld van clustering waarbij de blauwe kruisjes de gevonden punten zijn en de rode kruisjes het middelpunt van elke cluster.

<img src="../.images/IntroductieDeepLearning/clustering.jpg" width="400"/>

Bij de detectie speelt de <b>drempelwaarde</b> ook een belangrijke rol. Deze drempelwaarde bepaalt voor welke waarden van de uitvoer we de invoer als een stoma beschouwen. Als we bijvoorbeeld 0.5 als drempelwaarde nemen zal alle uitvoer groter dan 0.5 als "Stoma" beschouwd worden en alle uitvoer kleiner dan 0.5 als "Geen stoma". 

Een belangrijke afweging die je moet maken bij het kiezen van de drempelwaarde is de verhouding tussen <b>precision</b> en <b>recall</b>. 

<ul>
    <li>Precision: het percentage van de gevonden stomata dat ook daadwerkelijk stomata zijn.</li>
    <li>Recall: het percentage van het aantal stomata op een afbeelding dat daadwerkelijk gevonden werd.</li>
</ul>

Het is duidelijk dat een lage drempelwaarde zal zorgen voor een lage precision (er worden meer punten al stomata beschouwd, dus ook foute punten) maar een hoge recall (er gaan ook meer echte stomata gevonden worden). Andersom geldt dit ook, een hoge drempelwaarde zal zorgen voor een hoge precision maar lage recall.

De volgende code-cel zal voor drie afbeeldingen de stomata detecteren met het referentienetwerk. Je kan de drempelwaarde (thr) aanpassen en het resultaat interpreteren.

In [ ]:
diep_neuraal_netwerk.vind_stomata()

<div style='color: #690027;' markdown="1">
    <h2>3. Adversarial learning</h2> 
</div>

In het vakgebied van adversarial learning wordt er gezocht naar invoer die een machine kan misleiden en die dus een andere uitvoer oplevert dan de verwachte uitvoer. We kunnen dit zelf toepassen op ons eigen netwerk. Ons netwerk is getraind om stomata te herkennen op een microfoto van een blad. Dus wanneer we een afbeelding invoeren waarop iets volledig anders te zien is, zouden we verwachten dat het netwerk geen enkele stoma vindt. Toch blijkt dit niet het geval, voer maar eens volgende code-cel uit om te zien hoe het netwerk reageert op een afbeelding van een kat.

In [ ]:
diep_neuraal_netwerk.misleid_netwerk()

Je kan zien dat het netwerk niet echt leert wat een stoma is maar gewoon patronen leert herkennen. De ogen van de kat hebben ongeveer dezelfde vorm als een stoma en zullen dus ook als stoma gedetecteerd worden.

Er bestaan tal van voorbeelden van dit soort 'adversarial afbeeldingen'. Eén gebied in adverserial learning gaat op zoek naar afbeeldingen waarmee een (of meerdere) netwerken het moeilijk hebben. Een gekend voorbeeld zijn de afbeeldingen van chihuahua's en muffins waarbij een mens soms al twee keer moet kijken om het onderscheid te maken.

<img src="../.images/IntroductieDeepLearning/chihuahua_muffin.jpg" width="400"/>

Een andere categorie van adverserial learning zijn de zichtbare manipulaties. In deze categorie is het onderscheid voor de mens duidelijk te zien, maar kan een machine wel misleid worden. Denk hierbij bijvoorbeeld aan een zelfrijdende auto die een mens niet meer herkent omdat de mens bepaalde make-up heeft aangebracht. Onderzoekers zijn er ook in geslaagd om een netwerk voor gezichtsherkenning te misleiden door specifieke patronen op een brilmontuur te plaatsen. Zo konden ze het netwerk laten denken dat het een bepaalde beroemdheid zag ([Accessorize to a Crime: Real and Stealthy Attacks onState-of-the-Art Face Recognition](https://www.cs.cmu.edu/~sbhagava/papers/face-rec-ccs16.pdf)). Volgende afbeelding toont bovenaan de test personen met de speciaal ontworpen bril, onderaan staan de personen die het netwerk denkt te zien.

<img src="../.images/IntroductieDeepLearning/adversarial_glasses.jpg" width="400"/>

Een laatste categorie zijn de onzichtbare manipulaties. Het is mogelijk een afbeelding zo te manipuleren dat het voor de mens dezelfde afbeelding lijkt maar dat een neuraal netwerk een totaal andere conclussie trekt. In een paper ([Explaining and Harnessing Adversarial Examples](https://arxiv.org/abs/1412.6572)) hebben enkele onderzoekers een afbeelding van een panda aangepast door ruiswaarden toe te voegen. Een diep neuraal netwerk dat afbeeldingen classificeert, was voor de aanpassing 57,7% zeker dat er een panda op de afbeelding stond, na de aanpassing oordeelde het netwerk met 99,3% zekerheid dat het om een gibbon ging.

<img src="../.images/IntroductieDeepLearning/adversarial_panda.jpg"/>

Deze techniek kan ook misbruikt worden, denk maar aan een zelfrijdende auto die verkeersborden herkent met een diep neuraal netwerk. Iemand met kwade bedoelingen zou een normaal stopbord kunnen vervangen door een aangepast stopbord dat er gelijkaardig uitziet voor de mens maar niet kan gedetecteerd worden door het neurale netwerk van de auto. De auto zou dan alle stopborden negeren en levensgevaarlijk zijn voor de gebruiker.

<img src="../.images/cclic.png" alt="Banner" align="left" style="width:80px;"/><br><br>
Notebook KIKS, zie <a href="http://www.aiopschool.be">ai op school</a>, van F. wyffels, A. Meheus, T. Neutens & N. Gesquière is in licentie gegeven volgens een <a href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Naamsvermelding-NietCommercieel-GelijkDelen 4.0 Internationaal-licentie</a>. 